#### Haemophilus

In [16]:
import pandas as pd 
import numpy as np
import scipy
from scipy.stats import pearsonr
from sklearn import preprocessing
import scipy.sparse as sp
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, accuracy_score
from sklearn.linear_model import ElasticNet
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn import cross_validation
import xlsxwriter
from sklearn import linear_model, grid_search, datasets
from operator import itemgetter

import warnings
warnings.filterwarnings('ignore')

## Other disease Wikipedia Data

wiki_offset = [21,19,17,15,13,11,9,7,5,3,1,0]
gt_offset = [2,1,0]

## 21 normal offset demek 0. gun yani
## 2 normal offset demek 0. hafta yani

#wiki_offset = [21,17]
#gt_offset = [1]

workbook = xlsxwriter.Workbook('Haemophilus_Results.xlsx')
worksheet = workbook.add_worksheet()
col = 0
row = 0

for wiki_offset_day in wiki_offset:
    for gt_offset_day in gt_offset:
        col = 0
        %cd /home/tanlab1/Desktop/MultipleDisease/sonsonson/correlations/combined/LAST_OFFSET_WORK/Other_disease_WIKI_161_week/

        wiki_train_data = pd.read_csv("Haemophilus.csv")
        if wiki_offset_day ==21:
            ending_day = len(wiki_train_data)
        else:
            ending_day = wiki_offset_day - 21
            
        weekly_data = wiki_train_data[wiki_offset_day:ending_day].groupby(np.arange(len(wiki_train_data[wiki_offset_day:ending_day]))//7).sum()
        article_names = ['Haemophilus',
        'Haemophilus_influenzae',
        'Gram-negative_bacteria',
        'Facultative_anaerobic_organism',
        'Epiglottitis',
        'Cellulitis',
        'Meningitis',
        'Pathogenic_bacteria',
        'Hib_vaccine',
        'Osteomyelitis'
        ]
        selected_wiki_articles = ['Haemophilus','Haemophilus_influenzae','Gram-negative_bacteria',
                                  'Facultative_anaerobic_organism','Epiglottitis','Hib_vaccine','Osteomyelitis']
        other_disease_normalized_weekly_wiki_data = pd.DataFrame()
        for i in weekly_data:
            # Create x, where x the 'scores' column's values as floats
            x = weekly_data[i].values.astype(float)
            # Create a minimum and maximum processor object
            min_max_scaler = preprocessing.MinMaxScaler()
            # Create an object to transform the data to fit minmax processor
            x_scaled = min_max_scaler.fit_transform(x)
            # Run the normalizer on the dataframe
            df_normalized = pd.DataFrame(x_scaled)
            other_disease_normalized_weekly_wiki_data = pd.concat([other_disease_normalized_weekly_wiki_data, df_normalized], axis=1)
        other_disease_normalized_weekly_wiki_data.columns=article_names
        other_disease_normalized_weekly_wiki_data = other_disease_normalized_weekly_wiki_data[selected_wiki_articles]

        ## Other disease Google Trend Dataset

        %cd /home/tanlab1/Desktop/MultipleDisease/sonsonson/correlations/combined/LAST_OFFSET_WORK/Other_Disease_GT_normalized_csv_160_week/
        gt_train_data = pd.read_csv("haemophilus_normalized.csv",header=None)
        if gt_offset_day ==2:
            ending_day = len(gt_train_data)
        else:
            ending_day = gt_offset_day - 2
            
        gt_train_data = gt_train_data[gt_offset_day:ending_day]

        ## Flu disease Wiki Dataset

        flu_article_names = ['en-Common_cold',
                             'en-Human_flu',
                             'en-Influenza',
                             'en-Influenza-like_illness',
                             'en-Influenza_A_virus',
                             'en-Influenza_A_virus_subtype_H1N1',
                             'en-Influenzavirus_A',
                             'en-Influenzavirus_C',
                             'en-Oseltamivir',
                             'en-Zanamivir']

        %cd /home/tanlab1/Desktop/MultipleDisease/sonsonson/correlations/combined/LAST_OFFSET_WORK/Flu_WIKI_161_week/
        flu_wiki_data = pd.read_csv("top10_Flu_daily_wiki_data_161_week.csv")
        
        if wiki_offset_day ==21:
            ending_day = len(flu_wiki_data)
        else:
            ending_day = wiki_offset_day - 21
        
        flu_weekly_wiki_data = flu_wiki_data[wiki_offset_day:ending_day].groupby(np.arange(len(flu_wiki_data[wiki_offset_day:ending_day]))//7).sum()
        normalized_flu_weekly = pd.DataFrame()
        for i in flu_weekly_wiki_data:
            # Create x, where x the 'scores' column's values as floats
            x = flu_weekly_wiki_data[i].values.astype(float)
            # Create a minimum and maximum processor object
            min_max_scaler = preprocessing.MinMaxScaler()
            # Create an object to transform the data to fit minmax processor
            x_scaled = min_max_scaler.fit_transform(x)
            # Run the normalizer on the dataframe
            df_normalized = pd.DataFrame(x_scaled)
            normalized_flu_weekly = pd.concat([normalized_flu_weekly, df_normalized], axis=1)
        normalized_flu_weekly.columns=flu_article_names


        ## Flu disease Google Flu Trend Dataset

        %cd /home/tanlab1/Desktop/MultipleDisease/sonsonson/correlations/combined/LAST_OFFSET_WORK/Flu_GFT_DATA_160_week/
        flu_google_flu_trend_data = pd.read_csv("GFT_160week.csv")
       
        if gt_offset_day ==2:
            ending_day = len(flu_google_flu_trend_data)
        else:
            ending_day = gt_offset_day - 2
        flu_google_flu_trend_data = flu_google_flu_trend_data[gt_offset_day:ending_day]

        ## Combination of all training datasets

        combined_all_datasets = pd.DataFrame()
        other_disease_normalized_weekly_wiki_data = other_disease_normalized_weekly_wiki_data.reset_index()
        gt_train_data = gt_train_data.reset_index()
        normalized_flu_weekly = normalized_flu_weekly.reset_index()
        flu_google_flu_trend_data = flu_google_flu_trend_data.reset_index()
        combined_all_datasets = pd.concat([other_disease_normalized_weekly_wiki_data, gt_train_data, normalized_flu_weekly, flu_google_flu_trend_data], axis=1)
        del combined_all_datasets['index']

        ## Observed Data Set

        %cd /home/tanlab1/Desktop/MultipleDisease/sonsonson/correlations/combined/LAST_OFFSET_WORK/
        observation_data = pd.read_csv("ALL_OBSERVATION_DATA.csv")
        selected_observation_disease = ['Flu', 'Haemophilus']
        other_disease_normalized_weekly_wiki_data = observation_data[selected_observation_disease]
        observation_data = observation_data[selected_observation_disease]

        combined_all_datasets = combined_all_datasets.as_matrix()
        observation_data = observation_data.as_matrix()

        r2,mse,mae,best_scores = getScore(combined_all_datasets, observation_data,2)
        
        real_wiki_offset = wiki_offset_day - 21
        real_gt_offset = gt_offset_day - 2
        offset_value = str(real_wiki_offset) + "," + str(real_gt_offset)
        worksheet.write(row,col,str(offset_value))
        col = col + 1
        worksheet.write(row,col,str(r2[0::2]))
        col = col + 1
        worksheet.write(row,col,str(r2[1::2]))
        col = col + 1
        worksheet.write(row,col,str(sum(r2[0::2])/len(r2[0::2])))
        col = col + 1
        worksheet.write(row,col,str(sum(r2[1::2])/len(r2[1::2])))
        col = col + 1
        worksheet.write(row,col,str(mse[0::2]))
        col = col + 1
        worksheet.write(row,col,str(mse[1::2]))
        col = col + 1
        worksheet.write(row,col,str(sum(mse[0::2])/len(mse[0::2])))
        col = col + 1
        worksheet.write(row,col,str(sum(mse[1::2])/len(mse[1::2])))
        col = col + 1
        worksheet.write(row,col,str(mae[0::2]))
        col = col + 1
        worksheet.write(row,col,str(mae[1::2]))
        col = col + 1
        worksheet.write(row,col,str(sum(mae[0::2])/len(mae[0::2])))
        col = col + 1
        worksheet.write(row,col,str(sum(mae[1::2])/len(mae[1::2])))
        col = col + 1
        worksheet.write(row,col,str(best_scores))
        row = row + 1
workbook.close()

/home/tanlab1/Desktop/MultipleDisease/sonsonson/correlations/combined/LAST_OFFSET_WORK/Other_disease_WIKI_161_week
/home/tanlab1/Desktop/MultipleDisease/sonsonson/correlations/combined/LAST_OFFSET_WORK/Other_Disease_GT_normalized_csv_160_week
/home/tanlab1/Desktop/MultipleDisease/sonsonson/correlations/combined/LAST_OFFSET_WORK/Flu_WIKI_161_week
/home/tanlab1/Desktop/MultipleDisease/sonsonson/correlations/combined/LAST_OFFSET_WORK/Flu_GFT_DATA_160_week
/home/tanlab1/Desktop/MultipleDisease/sonsonson/correlations/combined/LAST_OFFSET_WORK
/home/tanlab1/Desktop/MultipleDisease/sonsonson/correlations/combined/LAST_OFFSET_WORK/Other_disease_WIKI_161_week
/home/tanlab1/Desktop/MultipleDisease/sonsonson/correlations/combined/LAST_OFFSET_WORK/Other_Disease_GT_normalized_csv_160_week
/home/tanlab1/Desktop/MultipleDisease/sonsonson/correlations/combined/LAST_OFFSET_WORK/Flu_WIKI_161_week
/home/tanlab1/Desktop/MultipleDisease/sonsonson/correlations/combined/LAST_OFFSET_WORK/Flu_GFT_DATA_160_week

#### hepatitis_A

In [ ]:
import pandas as pd 
import numpy as np
import scipy
from scipy.stats import pearsonr
from sklearn import preprocessing
import scipy.sparse as sp
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, accuracy_score
from sklearn.linear_model import ElasticNet
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn import cross_validation
import xlsxwriter
from sklearn import linear_model, grid_search, datasets
from operator import itemgetter

import warnings
warnings.filterwarnings('ignore')

## Other disease Wikipedia Data

wiki_offset = [21,19,17,15,13,11,9,7,5,3,1,0]
gt_offset = [2,1,0]

## 21 normal offset demek 0. gun yani
## 2 normal offset demek 0. hafta yani

#wiki_offset = [21,17]
#gt_offset = [1]

workbook = xlsxwriter.Workbook('hepatitis_A_Results.xlsx')
worksheet = workbook.add_worksheet()
col = 0
row = 0

for wiki_offset_day in wiki_offset:
    for gt_offset_day in gt_offset:
        col = 0
        %cd /home/tanlab1/Desktop/MultipleDisease/sonsonson/correlations/combined/LAST_OFFSET_WORK/Other_disease_WIKI_161_week/

        wiki_train_data = pd.read_csv("hepatitis_A.csv")
        if wiki_offset_day ==21:
            ending_day = len(wiki_train_data)
        else:
            ending_day = wiki_offset_day - 21
            
        weekly_data = wiki_train_data[wiki_offset_day:ending_day].groupby(np.arange(len(wiki_train_data[wiki_offset_day:ending_day]))//7).sum()
        article_names = ['Hepatitis',
'Hepatitis_A#Virology',
'Jaundice',
'Hepatitis_B',
'Hepatitis_C',
'Hepatitis_A_vaccine',
'Viral_hepatitis',
'World_Hepatitis_Day',
'Picornavirales',
'Hepatitis_A']
        selected_wiki_articles = [
'Hepatitis_A#Virology',
'Hepatitis_A_vaccine',
'Viral_hepatitis',
'World_Hepatitis_Day',
'Picornavirales',
'Hepatitis_A'  
        ]
        other_disease_normalized_weekly_wiki_data = pd.DataFrame()
        for i in weekly_data:
            # Create x, where x the 'scores' column's values as floats
            x = weekly_data[i].values.astype(float)
            # Create a minimum and maximum processor object
            min_max_scaler = preprocessing.MinMaxScaler()
            # Create an object to transform the data to fit minmax processor
            x_scaled = min_max_scaler.fit_transform(x)
            # Run the normalizer on the dataframe
            df_normalized = pd.DataFrame(x_scaled)
            other_disease_normalized_weekly_wiki_data = pd.concat([other_disease_normalized_weekly_wiki_data, df_normalized], axis=1)
        other_disease_normalized_weekly_wiki_data.columns=article_names
        other_disease_normalized_weekly_wiki_data = other_disease_normalized_weekly_wiki_data[selected_wiki_articles]

        ## Other disease Google Trend Dataset

        %cd /home/tanlab1/Desktop/MultipleDisease/sonsonson/correlations/combined/LAST_OFFSET_WORK/Other_Disease_GT_normalized_csv_160_week/
        gt_train_data = pd.read_csv("hepatitis_A_normalized.csv",header=None)
        if gt_offset_day ==2:
            ending_day = len(gt_train_data)
        else:
            ending_day = gt_offset_day - 2
            
        gt_train_data = gt_train_data[gt_offset_day:ending_day]

        ## Flu disease Wiki Dataset

        flu_article_names = ['en-Common_cold',
                             'en-Human_flu',
                             'en-Influenza',
                             'en-Influenza-like_illness',
                             'en-Influenza_A_virus',
                             'en-Influenza_A_virus_subtype_H1N1',
                             'en-Influenzavirus_A',
                             'en-Influenzavirus_C',
                             'en-Oseltamivir',
                             'en-Zanamivir']

        %cd /home/tanlab1/Desktop/MultipleDisease/sonsonson/correlations/combined/LAST_OFFSET_WORK/Flu_WIKI_161_week/
        flu_wiki_data = pd.read_csv("top10_Flu_daily_wiki_data_161_week.csv")
        
        if wiki_offset_day ==21:
            ending_day = len(flu_wiki_data)
        else:
            ending_day = wiki_offset_day - 21
        
        flu_weekly_wiki_data = flu_wiki_data[wiki_offset_day:ending_day].groupby(np.arange(len(flu_wiki_data[wiki_offset_day:ending_day]))//7).sum()
        normalized_flu_weekly = pd.DataFrame()
        for i in flu_weekly_wiki_data:
            # Create x, where x the 'scores' column's values as floats
            x = flu_weekly_wiki_data[i].values.astype(float)
            # Create a minimum and maximum processor object
            min_max_scaler = preprocessing.MinMaxScaler()
            # Create an object to transform the data to fit minmax processor
            x_scaled = min_max_scaler.fit_transform(x)
            # Run the normalizer on the dataframe
            df_normalized = pd.DataFrame(x_scaled)
            normalized_flu_weekly = pd.concat([normalized_flu_weekly, df_normalized], axis=1)
        normalized_flu_weekly.columns=flu_article_names


        ## Flu disease Google Flu Trend Dataset

        %cd /home/tanlab1/Desktop/MultipleDisease/sonsonson/correlations/combined/LAST_OFFSET_WORK/Flu_GFT_DATA_160_week/
        flu_google_flu_trend_data = pd.read_csv("GFT_160week.csv")
       
        if gt_offset_day ==2:
            ending_day = len(flu_google_flu_trend_data)
        else:
            ending_day = gt_offset_day - 2
        flu_google_flu_trend_data = flu_google_flu_trend_data[gt_offset_day:ending_day]

        ## Combination of all training datasets

        combined_all_datasets = pd.DataFrame()
        other_disease_normalized_weekly_wiki_data = other_disease_normalized_weekly_wiki_data.reset_index()
        gt_train_data = gt_train_data.reset_index()
        normalized_flu_weekly = normalized_flu_weekly.reset_index()
        flu_google_flu_trend_data = flu_google_flu_trend_data.reset_index()
        combined_all_datasets = pd.concat([other_disease_normalized_weekly_wiki_data, gt_train_data, normalized_flu_weekly, flu_google_flu_trend_data], axis=1)
        del combined_all_datasets['index']

        ## Observed Data Set

        %cd /home/tanlab1/Desktop/MultipleDisease/sonsonson/correlations/combined/LAST_OFFSET_WORK/
        observation_data = pd.read_csv("ALL_OBSERVATION_DATA.csv")
        selected_observation_disease = ['Flu', 'Hepatitis_A']
        other_disease_normalized_weekly_wiki_data = observation_data[selected_observation_disease]
        observation_data = observation_data[selected_observation_disease]

        combined_all_datasets = combined_all_datasets.as_matrix()
        observation_data = observation_data.as_matrix()

        r2,mse,mae,best_scores = getScore(combined_all_datasets, observation_data,2)
        
        real_wiki_offset = wiki_offset_day - 21
        real_gt_offset = gt_offset_day - 2
        offset_value = str(real_wiki_offset) + "," + str(real_gt_offset)
        worksheet.write(row,col,str(offset_value))
        col = col + 1
        worksheet.write(row,col,str(r2[0::2]))
        col = col + 1
        worksheet.write(row,col,str(r2[1::2]))
        col = col + 1
        worksheet.write(row,col,str(sum(r2[0::2])/len(r2[0::2])))
        col = col + 1
        worksheet.write(row,col,str(sum(r2[1::2])/len(r2[1::2])))
        col = col + 1
        worksheet.write(row,col,str(mse[0::2]))
        col = col + 1
        worksheet.write(row,col,str(mse[1::2]))
        col = col + 1
        worksheet.write(row,col,str(sum(mse[0::2])/len(mse[0::2])))
        col = col + 1
        worksheet.write(row,col,str(sum(mse[1::2])/len(mse[1::2])))
        col = col + 1
        worksheet.write(row,col,str(mae[0::2]))
        col = col + 1
        worksheet.write(row,col,str(mae[1::2]))
        col = col + 1
        worksheet.write(row,col,str(sum(mae[0::2])/len(mae[0::2])))
        col = col + 1
        worksheet.write(row,col,str(sum(mae[1::2])/len(mae[1::2])))
        col = col + 1
        worksheet.write(row,col,str(best_scores))
        row = row + 1
workbook.close()

/home/tanlab1/Desktop/MultipleDisease/sonsonson/correlations/combined/LAST_OFFSET_WORK/Other_disease_WIKI_161_week
/home/tanlab1/Desktop/MultipleDisease/sonsonson/correlations/combined/LAST_OFFSET_WORK/Other_Disease_GT_normalized_csv_160_week
/home/tanlab1/Desktop/MultipleDisease/sonsonson/correlations/combined/LAST_OFFSET_WORK/Flu_WIKI_161_week
/home/tanlab1/Desktop/MultipleDisease/sonsonson/correlations/combined/LAST_OFFSET_WORK/Flu_GFT_DATA_160_week
/home/tanlab1/Desktop/MultipleDisease/sonsonson/correlations/combined/LAST_OFFSET_WORK


In [1]:
def getScore(X,y, numberOfDisease):
    data = X
    gt = y
    r2 = []
    mse = []
    mae = []
    rep = []
    parameters = {"alpha": [1, 0.1, 0.01, 0.001, 0.0001],
                  "l1_ratio": [0, 0.25, 0.5,0.75,1],
              "fit_intercept": [True],
              "max_iter":[1000],
              "normalize":[True, False],
              "selection":['cyclic', 'random'],
              "tol":[0.0001, 0.001]}
    
    #cv = cross_validation.KFold(len(train), n_folds=10, shuffle=True)
    #for data,gt in cv:
       #print data, gt
    s = 0
    for random_generator in range(1,11):
        X_train, X_test, y_train, y_test = cross_validation.train_test_split(data, gt, test_size=0.3, random_state=random_generator)
        mtl = linear_model.MultiTaskElasticNet()
        grid_search = GridSearchCV(mtl, param_grid=parameters, n_jobs=-1)
        grid_search.fit(X_train, y_train)
        predictions = grid_search.predict(X_test)
        for i in range(0,numberOfDisease):
            s += 1
            r2.append(r2_score(y_test[:,i], predictions[:,i]))
            mse.append(mean_squared_error(y_test[:,i], predictions[:,i]))
            mae.append(mean_absolute_error(y_test[:,i], predictions[:,i]))

        temp = report(grid_search.grid_scores_)
        rep.append(temp)
    return r2, mse, mae, rep

In [2]:
def report(grid_scores, n_top=1):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    l = []
    for i, score in enumerate(top_scores):
        
        #print("Model with rank: {0}".format(i + 1))
        #print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
        #     score.mean_validation_score,
        #      np.std(score.cv_validation_scores)))
        #print("Parameters: {0}".format(score.parameters))
        #print("")
        l.append(score.mean_validation_score)
        l.append(np.std(score.cv_validation_scores))
        l.append(score.parameters)
    return l